In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn
from model import BN
import torch.optim as optim
import torch.nn.functional  as F
from torchsummary import summary

In [2]:
train_datasets=datasets.CIFAR10(root= './data_CIFAR10', 
                                transform=transforms.Compose([transforms.ToTensor()]),
                                train=True, download=True)
test_datasets = datasets.CIFAR10(root='./data_CIFAR10', 
                                 transform = transforms.Compose([transforms.ToTensor()]),
                                 train=False, download = True)
train_loader = DataLoader(dataset = train_datasets, batch_size=128, shuffle=True, num_workers=5, pin_memory=True)
test_loader = DataLoader(dataset = test_datasets, batch_size=128, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
from tqdm import tqdm
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def train(device, model, train_loader, optimizer, epoch):
    model.train() # access model in train model
    losses=[]
    accuracies=[]
    pbar = tqdm(train_loader)
    for idx , data in enumerate(pbar):
        input , label = data
        input.to(device)
        label.to(device) 
        optimizer.zero_grad()
        output = model(input)
        pred = torch.argmax(output,dim=1)
        acc=pred.eq(label).sum()/input.shape[0]
        accuracies.append(acc)
        # print(output.shape)
        loss  = F.nll_loss(output,label)
        loss.backward()
        optimizer.step()
        losses.append(loss)
    avg_loss = sum(losses)/len(losses)
    avg_acc = sum(accuracies)/len(accuracies)
    print (f'at epoch:{epoch} avg_loss:{avg_loss},avg_train_acc:{avg_acc}')

def test(device, model, test_loader):
    model.eval() # access model in evaluation mode # set drop out and BN off
    accuracies=[]
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            input , label = data
            input.to(device)
            label.to(device)
            out = model(input)
            pred = torch.argmax(out, dim=1)
            acc= pred.eq(label).sum()/input.shape[0]
            accuracies.append(acc)
        avg_acc = np.mean(accuracies)
        print (f'avg_test_acc:{avg_acc}')

        
    

In [10]:
model = BN()
model.to(device)
summary(model, input_size=(3,32,32))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 30, 30]             448
              ReLU-2           [-1, 16, 30, 30]               0
       BatchNorm2d-3           [-1, 16, 30, 30]              32
            Conv2d-4           [-1, 16, 28, 28]           2,320
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
            Conv2d-7           [-1, 16, 26, 26]           2,320
              ReLU-8           [-1, 16, 26, 26]               0
       BatchNorm2d-9           [-1, 16, 26, 26]              32
        MaxPool2d-10           [-1, 16, 13, 13]               0
           Conv2d-11           [-1, 16, 11, 11]           2,320
             ReLU-12           [-1, 16, 11, 11]               0
      BatchNorm2d-13           [-1, 16, 11, 11]              32
           Conv2d-14             [-1, 1

/Users/chourasi/ERA/S5/ERA/S8/model.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [11]:
epochs=20
optimizer = optim.SGD(model.parameters(), lr=.01, momentum=0.9)
for e in range(epochs):
    train(device, model, train_loader, optimizer, e)
    test(device, model,test_loader)

100%|██████████| 391/391 [01:14<00:00,  5.23it/s]


at epoch:0 avg_loss:1.5708646774291992,avg_train_acc:0.42879635095596313
avg_test_acc:0.4756724536418915


100%|██████████| 391/391 [01:15<00:00,  5.16it/s]


at epoch:1 avg_loss:1.2035819292068481,avg_train_acc:0.5715792775154114
avg_test_acc:0.5825751423835754


100%|██████████| 391/391 [01:14<00:00,  5.22it/s]


...

at epoch:17 avg_loss:0.7824083566665649,avg_train_acc:0.7225543260574341
avg_test_acc:0.6850277185440063


100%|██████████| 391/391 [01:12<00:00,  5.41it/s]


at epoch:18 avg_loss:0.7763996720314026,avg_train_acc:0.7261629104614258
avg_test_acc:0.6873022317886353


100%|██████████| 391/391 [01:11<00:00,  5.49it/s]


at epoch:19 avg_loss:0.7722100019454956,avg_train_acc:0.7293718457221985
avg_test_acc:0.6793908476829529
